In [8]:
 %cd /content/drive/MyDrive/Voice_Classification

/content/drive/MyDrive/Voice_Classification


Reference: https://github.com/jiaaro/pydub

In [2]:
# Install requirements

!pip install -r requirements.txt

  Preparing metadata (setup.py) ... done
  Created wheel for ffmpeg: filename=ffmpeg-1.4-py3-none-any.whl size=6080 sha256=0327d436c88346c5e38b2d1b804b4be8a7392cfb6d6e36182eab3b0bbba604a3
  Stored in directory: /root/.cache/pip/wheels/8e/7a/69/cd6aeb83b126a7f04cbe7c9d929028dc52a6e7d525ff56003a
Successfully built ffmpeg


In [3]:
# Imports

import pydub
import os
from pathlib import Path
import matplotlib.pyplot as plt
import shutil


In [ ]:
voice1 = pydub.AudioSegment.from_file("raw_data/Nahid.ogg")

In [ ]:
voice1

In [ ]:
# Export file

voice1.export("Nahid.mp3")

In [ ]:
# Merge multi file of one class

audios_files = "./raw_data"
merged_files_directory = "./merged_audios"
shutil.rmtree(merged_files_directory)

Path(merged_files_directory).mkdir(parents=True, exist_ok=True)

for audio in os.listdir(audios_files):
  voice_file1 = os.path.join(audios_files, audio)

  print(voice_file1)
  if "_" in Path(audio).stem:
    if Path(audio).stem.split("_")[1] == '1':
      voice_file2 = voice_file1.replace('1','2')
      voice1 = pydub.AudioSegment.from_file(voice_file1)
      voice2 = pydub.AudioSegment.from_file(voice_file2)

      merged_voice = voice1 + voice2
      merged_voice.export(os.path.join( merged_files_directory, audio.split("_")[0] + ".ogg"), format="ogg", bitrate="192k")
  else:
    voice = pydub.AudioSegment.from_file(voice_file1)
    voice.export(os.path.join(merged_files_directory, audio), format="ogg", bitrate="192k")
    print(os.path.join(merged_files_directory, audio))

In [ ]:
# Plot voice

voice_1 = pydub.AudioSegment.from_file("merged_audios/Nahid.ogg")
print(type(voice_1))

voice_2 = voice_1.get_array_of_samples()
print(type(voice_2))

plt.figure(figsize=(20, 2))
plt.plot(voice_2)
plt.show()

In [ ]:
# Preprocessing: Remove silence from audios
"""
split_on_silence: remove silence from audios
"""

sound = pydub.AudioSegment.from_file("./merged_audios/Nahid.ogg")
audio_chunks = pydub.silence.split_on_silence(sound, min_silence_len=2000, silence_thresh=-45)
audio_chunks

In [ ]:
merged_audio = sum(audio_chunks)
merged_audio


In [ ]:
audios_raw_dir_path = "./merged_audios"
audios_remove_silence_dir_path = "./wav_data"
shutil.rmtree(audios_remove_silence_dir_path)
Path(audios_remove_silence_dir_path).mkdir(exist_ok=True)

for file in os.listdir(audios_raw_dir_path):
    file_path = os.path.join(audios_raw_dir_path, file)
    audio = pydub.AudioSegment.from_file(file_path)

    # draw
    print(file)
    samples = audio.get_array_of_samples()
    plt.figure(figsize=(20, 2), dpi=80)
    plt.plot(samples)
    plt.show()

    # 32 bit to 16 bit
    audio = audio.set_sample_width(2)
    audio = audio.set_frame_rate(48000)
    audio = audio.set_channels(1)

    # remove silence
    audio_chunks = pydub.silence.split_on_silence(audio, min_silence_len = 2000, silence_thresh = -45)
    combined_audio = sum(audio_chunks)
    filename = Path(file_path).stem
    output_file_path = os.path.join(audios_remove_silence_dir_path, filename + ".wav")
    combined_audio.export(output_file_path, format="wav", bitrate="32k")

In [ ]:
shutil.rmtree("./dataset")
for file in os.listdir("./wav_data"):
    file_path = os.path.join("./wav_data", file)
    audio = pydub.AudioSegment.from_file(file_path)
    chunks = pydub.utils.make_chunks(audio, 1000)
    os.makedirs(os.path.join("./dataset", Path(file).stem), exist_ok=True)
    for i, chunk in enumerate(chunks):
      if len(chunk) >= 1000:
        print(os.path.join("./dataset", Path(file).stem, f"voice_{i}.wav"))
        chunk.export(os.path.join("./dataset", Path(file).stem, f"voice_{i}.wav"), format="wav")
